# Structured Outputs with Azure OpenAI
## Using Azure OpenAI's native Structured Outputs for guaranteed JSON

This notebook demonstrates **Structured Outputs** using Azure OpenAI's API.

**What is it:** Azure OpenAI can guarantee that responses match a Pydantic schema exactly - no more JSON parsing errors!

**Use case:** Extract structured product reviews into guaranteed valid JSON format.

## Installation

In [ ]:
# Install required packages
!pip install openai pydantic python-dotenv

## Setup

In [ ]:
import os
from enum import Enum
from pydantic import BaseModel, Field
from typing import List

# Load from .env file
from dotenv import load_dotenv
load_dotenv()

# Verify API key is set
if not os.getenv("AZURE_OPENAI_API_KEY") or not os.getenv("AZURE_OPENAI_ENDPOINT") or not os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"):
    raise ValueError("Please set AZURE_OPENAI_API_KEY, AZURE_OPENAI_ENDPOINT, AZURE_OPENAI_DEPLOYMENT_NAME environment variable")

print("✅ Setup complete!")

## Define the Output Schema

We'll extract product reviews into a structured format.

In [ ]:
class Sentiment(str, Enum):
    """Review sentiment - must be one of these exact values"""
    POSITIVE = "positive"
    NEGATIVE = "negative"
    NEUTRAL = "neutral"

class Category(str, Enum):
    """Product category"""
    ELECTRONICS = "electronics"
    CLOTHING = "clothing"
    FOOD = "food"
    BOOKS = "books"
    OTHER = "other"

class Review(BaseModel):
    """Structured product review"""
    product_name: str = Field(description="Name of the product")
    category: Category = Field(description="Product category")
    sentiment: Sentiment = Field(description="Overall sentiment")
    rating: int = Field(ge=1, le=5, description="Rating from 1-5")
    pros: List[str] = Field(description="List of positive aspects")
    cons: List[str] = Field(description="List of negative aspects")
    would_recommend: bool = Field(description="Would recommend to others")

print("Schema defined!")
print(f"\nExample of valid JSON structure:")
print(Review(
    product_name="Wireless Headphones",
    category=Category.ELECTRONICS,
    sentiment=Sentiment.POSITIVE,
    rating=4,
    pros=["Great sound quality", "Comfortable"],
    cons=["Battery life could be better"],
    would_recommend=True
).model_dump_json(indent=2))

## Sample Review Text

Let's use some example product reviews.

In [ ]:
sample_reviews = [
    """
    I recently purchased the Sony WH-1000XM5 headphones and I'm absolutely blown away! 
    The noise cancellation is incredible - I can work in a busy coffee shop and hear nothing 
    but my music. The sound quality is pristine with deep bass and clear highs. They're also 
    super comfortable for long listening sessions. My only complaint is that they're a bit 
    expensive and the battery life, while good at 30 hours, isn't quite as impressive as 
    some competitors. Overall, definitely worth the investment for audiophiles!
    """,
    
    """
    Bought this winter jacket from North Face and it's been disappointing. The material feels 
    cheap for the price point - I expected better quality. It keeps me warm in mild cold, but 
    when temperatures really drop, I need extra layers. The zipper got stuck twice already in 
    the first month. On the plus side, it looks stylish and has plenty of pockets. But for 
    $200, I expected much better durability. Would not buy again.
    """,
    
    """
    This cookbook "Salt, Fat, Acid, Heat" by Samin Nosrat is a game-changer! Unlike typical 
    recipe books, it teaches you the WHY behind cooking, not just the how. The illustrations 
    are beautiful and the explanations are clear. I've become a much better cook since reading 
    it. It's a bit text-heavy and might overwhelm beginners, but if you want to truly understand 
    cooking, this is the book. Highly recommend to anyone serious about improving their skills!
    """
]

print(f"Loaded {len(sample_reviews)} sample reviews")

## Method 1: Without Constrained Decoding (Just Prompting)

First, let's see what happens with pure prompting - **no guarantees** of valid JSON.

In [ ]:
from openai import AzureOpenAI
import json
import os

client = AzureOpenAI(
    api_version="2024-12-01-preview",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

def extract_review_without_constraints(review_text: str) -> dict:
    """Extract review using pure prompting - might fail!"""
    
    prompt = f"""
    Extract the following information from the product review and return it as JSON:
    - product_name (string)
    - category (one of: electronics, clothing, food, books, other)
    - sentiment (one of: positive, negative, neutral)
    - rating (integer 1-5)
    - pros (list of strings)
    - cons (list of strings)
    - would_recommend (boolean)
    
    Review: {review_text}
    
    Return ONLY the JSON, no explanation.
    """
    
    response = client.chat.completions.create(
        model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3
    )
    
    return response.choices[0].message.content

# Test with first review
print("❌ WITHOUT CONSTRAINTS (might fail):")
print("=" * 60)

result = extract_review_without_constraints(sample_reviews[0])
print(result)

# Try to parse as JSON
try:
    parsed = json.loads(result)
    print("\n✅ Valid JSON!")
    
    # But check if it matches our schema
    try:
        Review(**parsed)
        print("✅ Matches Review schema!")
    except Exception as e:
        print(f"⚠️ Valid JSON but doesn't match schema: {e}")
        
except json.JSONDecodeError as e:
    print(f"❌ Invalid JSON: {e}")

## Method 2: With Structured Outputs (Guaranteed Valid JSON)

Now let's use **Azure OpenAI's native Structured Outputs** to **guarantee** valid JSON that matches our schema.

### How it works:
1. We pass the Pydantic schema to OpenAI via `response_format`
2. The model's output is constrained to match the schema exactly
3. Azure OpenAI ensures the response is valid JSON matching the schema
4. Result: **100% guaranteed valid JSON**

This is similar to what `outlines` does, but built directly into Azure OpenAI!

In [ ]:
def extract_review_with_structured_outputs(review_text: str) -> Review:
    """Extract review using Azure OpenAI Structured Outputs - guaranteed valid!
       Azure OpenAI supports Structured Output,
       For OpenAI standard instance `outlines` should be used which does exactly the same."""
    
    prompt = f"""
    Extract structured information from this product review.
    
    Review: {review_text}
    """

    response = client.beta.chat.completions.parse(
        model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
        messages=[{"role": "user", "content": prompt}],
        response_format=Review, # Review format is declared as expected output to enforce proper JSON format
        temperature=0.3
    )
    
    return response.choices[0].message.parsed

print("✅ Function created with Structured Outputs support!")

In [ ]:
# No special prompt template needed - we can just use the function directly!
print("✅ Ready to extract reviews with guaranteed valid JSON!")

In [ ]:
print("✅ WITH STRUCTURED OUTPUTS (guaranteed valid):")
print("=" * 60)

# Process first review with structured outputs
result = extract_review_with_structured_outputs(sample_reviews[0])

print(f"Type: {type(result)}")
print(f"\n{result.model_dump_json(indent=2)}")

# This is GUARANTEED to be a valid Review object
print(f"\n✅ Product: {result.product_name}")
print(f"✅ Category: {result.category}")
print(f"✅ Sentiment: {result.sentiment}")
print(f"✅ Rating: {result.rating}/5")
print(f"✅ Would recommend: {result.would_recommend}")

## Process All Reviews

Let's extract all reviews and see the guaranteed consistency.

In [ ]:
extracted_reviews = []

print("Processing all reviews...\n")

for i, review_text in enumerate(sample_reviews, 1):
    print(f"Processing review {i}/{len(sample_reviews)}...")
    
    result = extract_review_with_structured_outputs(review_text)
    extracted_reviews.append(result)
    
    print(f"  ✅ {result.product_name} ({result.category.value})")
    print(f"     Sentiment: {result.sentiment.value} | Rating: {result.rating}/5\n")

print(f"\n✅ Successfully extracted {len(extracted_reviews)} reviews!")
print("All results are GUARANTEED to match the Review schema.")

## Demonstrate the Guarantee

Let's try with ambiguous text to show it MUST follow the schema.

In [ ]:
# This is intentionally vague/weird text
weird_review = """
I bought something yesterday. It was okay I guess. Some things were good, others not so much.
I might tell my friend about it, or maybe not. It cost money.
"""

print("Testing with ambiguous review...\n")
print(f"Input: {weird_review}\n")

result = extract_review_with_structured_outputs(weird_review)

print("Output (still valid JSON matching schema!):")
print(result.model_dump_json(indent=2))

print("\n✅ Even with vague input, output is guaranteed valid!")
print(f"   Category MUST be one of: {[c.value for c in Category]}")
print(f"   Sentiment MUST be one of: {[s.value for s in Sentiment]}")
print(f"   Rating MUST be integer 1-5")

## Key Takeaways

### What `Azure OpenAI` Structured Outputs does:
1. ✅ Guarantees valid JSON structure
2. ✅ Enforces enum values (sentiment, category)
3. ✅ Enforces type constraints (rating must be int 1-5)
4. ✅ Ensures required fields are present
5. ✅ No parsing errors, ever

### How it works with `Azure OpenAI`:
- Uses the `response_format` parameter with Pydantic models
- The model's output is constrained to match the schema
- Built directly into Azure OpenAI API (no external libraries needed)
- Leverages the `.beta.chat.completions.parse()` method

### What `OpenAI` outlines does:
1. ✅ The same but using outlines library and slightly different code
2. ✅ Guarantees valid JSON code enforcing rules

**For complex business rules with custom validation logic**, see the next notebook on true logits masking with open-source models!

## Bonus: Comparison Test

Let's run the same review through both methods 10 times to see consistency.

In [ ]:
test_review = sample_reviews[0]
num_runs = 5

print("Running consistency test...\n")
print(f"Running each method {num_runs} times on the same review.\n")

# Without constraints
print("❌ WITHOUT CONSTRAINTS:")
without_failures = 0
for i in range(num_runs):
    try:
        result = extract_review_without_constraints(test_review)
        parsed = json.loads(result)
        Review(**parsed)
        print(f"  Run {i+1}: ✅ Valid")
    except Exception as e:
        without_failures += 1
        print(f"  Run {i+1}: ❌ Failed - {str(e)[:50]}")

print(f"\nFailure rate: {without_failures}/{num_runs} ({without_failures/num_runs*100:.0f}%)\n")

# With structured outputs
print("✅ WITH STRUCTURED OUTPUTS:")
for i in range(num_runs):
    result = extract_review_with_structured_outputs(test_review)
    print(f"  Run {i+1}: ✅ Valid (guaranteed)")

print(f"\nFailure rate: 0/{num_runs} (0%)")
print("\n🎯 Structured outputs provide 100% reliability!")